In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Modelos
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel, SelectKBest, mutual_info_classif
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score, confusion_matrix, make_scorer, roc_auc_score, classification_report
from sklearn.utils.class_weight import compute_class_weight


In [129]:
df = pd.read_csv("/workspaces/Final_Project_DataScient/data/processed/data_final.csv")
df

,MARITAL,RENTHOM1,VETERAN3,EMPLOY1,_AGEG5YR,_INCOMG1,_EDUCAG,_CHLDCNT,HHADULT,SEXVAR,...,_DRDXAR2,_PHYS14D,_MENT14D,_BMI5CAT,DECIDE,DIFFALON,LSATISFY,EMTSUPRT,SDHISOLT,tobacco_use
0,0.0,0.0,0.0,0.0,8.0,4.0,3.0,0.0,2,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,6.0,8.0,-1.0,3.0,0.0,4,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,6.0,2.0,1.0,0.0,2,1.0,...,1.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,6.0,5.0,2.0,0.0,2,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,7.0,8.0,3.0,2.0,0.0,2,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114301,1.0,0.0,0.0,1.0,8.0,5.0,2.0,0.0,1,0.0,...,1.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0
114302,0.0,1.0,0.0,0.0,9.0,5.0,3.0,0.0,4,0.0,...,1.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0
114303,0.0,0.0,1.0,0.0,6.0,4.0,3.0,1.0,2,1.0,...,0.0,1.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
114304,0.0,2.0,0.0,7.0,9.0,-1.0,1.0,0.0,1,1.0,...,0.0,2.0,2.0,0.0,-1.0,0.0,-1.0,0.0,1.0,2.0


In [130]:
columnas = df.columns
columnas
for col in columnas:
    df = df[df[col] != -1]

In [131]:
# 'df' es el DataFrame completo
X = df.drop(columns=['ADDEPEV3'])  # variables predictoras
y = df['ADDEPEV3']                 # variable objetivo

# Calculamos la info mutua de cada variable con y
mi = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

# Lo ponemos en un DataFrame para ordenar mejor
mi_df = pd.DataFrame({'feature': X.columns, 'mutual_info': mi})
mi_df = mi_df.sort_values(by='mutual_info', ascending=False)

print(mi_df)

        feature  mutual_info
30     _MENT14D     0.089999
32       DECIDE     0.044524
34     LSATISFY     0.039869
36     SDHISOLT     0.035134
29     _PHYS14D     0.029201
35     EMTSUPRT     0.027770
3       EMPLOY1     0.025877
14     SDHFOOD1     0.022657
27     _ASTHMS1     0.021851
28     _DRDXAR2     0.020277
9        SEXVAR     0.019405
20     _TOTINDA     0.017485
5      _INCOMG1     0.016339
33     DIFFALON     0.016162
18     SLEPTIM1     0.015096
17     SDHTRNSP     0.012864
16     SDHBILLS     0.011895
11     _HLTHPLN     0.011073
12     _AIDTST4     0.010556
0       MARITAL     0.009273
31     _BMI5CAT     0.008477
8       HHADULT     0.007649
15     SDHUTILS     0.007630
1      RENTHOM1     0.007087
37  tobacco_use     0.007071
6       _EDUCAG     0.006352
4      _AGEG5YR     0.005314
10     MEDCOST1     0.004130
22     CVDSTRK3     0.003472
23     DIABETE4     0.003109
24     COVIDPOS     0.002384
21     _RFBING6     0.002369
19     HIVRISK5     0.002202
13     SDHEMPL

In [132]:
X = df.drop(columns=['ADDEPEV3'])
y = df['ADDEPEV3']

model = RandomForestClassifier(random_state=42)
model.fit(X, y)

importances = model.feature_importances_

importances_df = pd.DataFrame({'feature': X.columns, 'importance': importances})
importances_df = importances_df.sort_values(by='importance', ascending=False)

print(importances_df)

        feature  importance
30     _MENT14D    0.112458
4      _AGEG5YR    0.066448
18     SLEPTIM1    0.061085
5      _INCOMG1    0.056206
32       DECIDE    0.050311
3       EMPLOY1    0.041513
8       HHADULT    0.039732
6       _EDUCAG    0.037147
31     _BMI5CAT    0.036491
0       MARITAL    0.035149
36     SDHISOLT    0.034380
34     LSATISFY    0.032886
29     _PHYS14D    0.031615
24     COVIDPOS    0.025975
37  tobacco_use    0.024220
7      _CHLDCNT    0.022456
23     DIABETE4    0.020245
14     SDHFOOD1    0.019997
9        SEXVAR    0.019817
25     COLNCNCR    0.019518
28     _DRDXAR2    0.019097
27     _ASTHMS1    0.018274
1      RENTHOM1    0.017541
12     _AIDTST4    0.017211
20     _TOTINDA    0.016842
35     EMTSUPRT    0.015537
21     _RFBING6    0.014187
33     DIFFALON    0.013858
2      VETERAN3    0.011780
26       _MICHD    0.011708
13     SDHEMPLY    0.009907
10     MEDCOST1    0.008064
16     SDHBILLS    0.007810
17     SDHTRNSP    0.007108
22     CVDSTRK3    0

In [ ]:
X = df.drop(columns=['ADDEPEV3'])
y = df['ADDEPEV3']

model = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X, y)
coef = model.coef_[0]
coef_df = pd.DataFrame({'feature': X.columns, 'coef': coef})
coef_df['abs_coef'] = np.abs(coef_df['coef'])
coef_df = coef_df.sort_values(by='abs_coef', ascending=False)
print(coef_df)


        feature      coef  abs_coef
30     _MENT14D  1.034702  1.034702
32       DECIDE  1.009903  1.009903
9        SEXVAR  0.703399  0.703399
36     SDHISOLT  0.523699  0.523699
11     _HLTHPLN  0.473394  0.473394
28     _DRDXAR2  0.455076  0.455076
19     HIVRISK5  0.371048  0.371048
12     _AIDTST4  0.310490  0.310490
34     LSATISFY  0.277372  0.277372
6       _EDUCAG  0.210453  0.210453
27     _ASTHMS1 -0.205640  0.205640
37  tobacco_use  0.171533  0.171533
31     _BMI5CAT  0.155662  0.155662
26       _MICHD  0.123394  0.123394
33     DIFFALON  0.112401  0.112401
20     _TOTINDA -0.099826  0.099826
7      _CHLDCNT -0.093414  0.093414
29     _PHYS14D  0.089962  0.089962
4      _AGEG5YR -0.085647  0.085647
25     COLNCNCR  0.083839  0.083839
1      RENTHOM1  0.081591  0.081591
2      VETERAN3  0.075864  0.075864
17     SDHTRNSP  0.071894  0.071894
18     SLEPTIM1  0.070706  0.070706
16     SDHBILLS -0.065455  0.065455
3       EMPLOY1  0.043842  0.043842
8       HHADULT -0.033828  0

In [148]:
top_features = ['_MENT14D', 'LSATISFY', '_PHYS14D', 'SLEPTIM1',
                '_INCOMG1', '_BMI5CAT', '_AGEG5YR', '_EDUCAG',
                'DECIDE', 'SDHISOLT', 'SEXVAR', 'HHADULT', 'DIFFALON', '_DRDXAR2','_CHLDCNT',
                'SDHTRNSP']

df_coded = pd.get_dummies(
    df, columns=["EMPLOY1"])
sel_model = SelectFromModel(
    LogisticRegression(
        penalty='l1',
        solver='liblinear',
        class_weight='balanced',
        max_iter=500,
        random_state=42
    )
)
sel_model.fit(
    df_coded[top_features + [col for col in df_coded.columns if
                             col.startswith('RENTHOM1_') or 
                             col.startswith('EMPLOY1_')]], 
    df_coded['ADDEPEV3']
)
selected_indices = sel_model.get_support(indices=True)
all_features = top_features + [col for col in df_coded.columns if 
                                   col.startswith('RENTHOM1_') or 
                                   col.startswith('EMPLOY1_')]
selected_features = [all_features[idx] for idx in selected_indices]

In [149]:
# Filtramos el DataFrame
X = df_coded[selected_features].copy()
y = df_coded['ADDEPEV3'].copy()

# ------------------------------------------------------------------------------
# 5. Filtrado final con SelectKBest (mutual info)
# ------------------------------------------------------------------------------
selector = SelectKBest(mutual_info_classif, k=20)
selector.fit(X, y)

selected_indices_k = selector.get_support(indices=True)
selected_features_k = X.columns[selected_indices_k]

X = X[selected_features_k]

In [150]:
# 6. División en entrenamiento y prueba
# ------------------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ------------------------------------------------------------------------------
# 7. SMOTE en el conjunto de entrenamiento
# ------------------------------------------------------------------------------
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

In [151]:
# 8. Cálculo de pesos de clases
# ------------------------------------------------------------------------------
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))
print("Pesos de clases :", class_weights)

# ------------------------------------------------------------------------------
# 9. Cálculo de scale_pos_weight para XGBoost
# ------------------------------------------------------------------------------
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print("Valor scale_pos_weight :", scale_pos_weight)

# ------------------------------------------------------------------------------
# 10. Parametros para CatBoost
# ------------------------------------------------------------------------------
param_catboost = {
    'depth': [4, 6, 8],
    'learning_rate': [0.05, 0.08, 0.2],
    'l2_leaf_reg': [1, 3, 5],
    'iterations': [200, 300, 500],
}

# Parametros para XGBoost
param_xgboost = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'lambda': [0, 0.5, 1],
    'alpha': [0, 0.5, 1],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 5, 10],
}


Pesos de clases : {np.float64(0.0): np.float64(0.6289594093042437), np.float64(1.0): np.float64(2.4385944875894623)}
Valor scale_pos_weight : 3.877188975178925


In [152]:
# 11. Busqueda de hyperparametros para CatBoost
# ------------------------------------------------------------------------------
X_train_sub, X_valid_sub, y_train_sub, y_valid_sub = train_test_split(
    X_train_bal, y_train_bal, 
    test_size=0.2, 
    stratify=y_train_bal, 
    random_state=42
)

In [153]:
# Busqueda de hyperparametros para CatBoost
cat_random = RandomizedSearchCV(
    estimator=CatBoostClassifier(
        eval_metric='Recall',
        early_stopping_rounds=20,
        verbose=0,
        random_state=42,
        class_weights=class_weights
    ),
    param_distributions=param_catboost,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
cat_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x78736b2dab90>,
                   n_jobs=-1,
                   param_distributions={'depth': [4, 6, 8],
                                        'iterations': [200, 300, 500],
                                        'l2_leaf_reg': [1, 3, 5],
                                        'learning_rate': [0.05, 0.08, 0.2]},
                   random_state=42, scoring='f1', verbose=1)

In [154]:
print("Mejores parámetros :", cat_random.best_params_)
print("Valor F1 :", cat_random.best_score_)

Mejores parámetros : {'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 300, 'depth': 4}
Valor F1 : 0.5624685239714735


In [155]:
final_catboost = CatBoostClassifier(
    learning_rate=0.05,
    l2_leaf_reg=3,
    iterations=300,
    depth=4,
    eval_metric='Recall',
    early_stopping_rounds=20,
    verbose=0,
    random_state=42,
    class_weights=class_weights
)
final_catboost.fit(X_train, y_train)


In [156]:
# Primero entren0 el modelo final con todos los datos de entrenamiento balanceados
model = final_catboost

# Después hago las predicciones en el conjunto de prueba
y_pred = model.predict_proba(X_test)[:, 1]

# Aplico un umbral personalizado
threshold = 0.55
y_pred_threshold = (y_pred >= threshold).astype(int)

print("Mejor modelo CatBoost:")
print("F1 :", f1_score(y_test, y_pred_threshold))
print("Recall :", recall_score(y_test, y_pred_threshold))
print("Precisión :", precision_score(y_test, y_pred_threshold))
print("Accuracy :", accuracy_score(y_test, y_pred_threshold))
print("Clasification Report:\n", classification_report(y_test, y_pred_threshold))

Mejor modelo CatBoost:
F1 : 0.5744118757076362
Recall : 0.6951887941534713
Precisión : 0.48938906752411576
Accuracy : 0.7887605369965657
Clasification Report:
               precision    recall  f1-score   support

         0.0       0.91      0.81      0.86     12731
         1.0       0.49      0.70      0.57      3284

    accuracy                           0.79     16015
   macro avg       0.70      0.75      0.72     16015
weighted avg       0.83      0.79      0.80     16015



In [143]:
# Busqueda de hyperparametros para XGBoost
xgb_random = RandomizedSearchCV(
    estimator=XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        eval_metric='logloss'
    ),
    param_distributions=param_xgboost,
    n_iter=30,
    scoring='recall',
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

xgb_random.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_cons...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None, ...),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'alpha': [0, 0.5, 1],
                                        'colsample_bytree': [0.7, 0.8, 0.9],
                                        'lambda': [0, 0.5, 1],
                                        'max_depth': [3, 5, 7],
                                        'min_child_weight': [1, 5, 10],
                                        'n_estimators': [100, 200, 300],
                                        'scale_pos_weight': [1, 5, 10],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='recall', verbose=1)

In [144]:
print("Mejores parámetros :", xgb_random.best_params_)
print("Valor F1 :", xgb_random.best_score_)

Mejores parámetros : {'subsample': 0.9, 'scale_pos_weight': 10, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 3, 'lambda': 0.5, 'colsample_bytree': 0.8, 'alpha': 1}
Valor F1 : 0.8803867823968327


In [145]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, classification_report

# 1. Definir el modelo con los mejores hiperparámetros
xgb_final = XGBClassifier(
    subsample=0.7,
    scale_pos_weight=10,
    n_estimators=200,
    min_child_weight=5,
    max_depth=3,
    reg_lambda=1,
    reg_alpha=0.5,
    colsample_bytree=0.8,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# 2. Entrenar con todos los datos balanceados
xgb_final.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [146]:
# Después de entrenar el modelo
model = xgb_final

# Obtener probabilidades en el conjunto de prueba
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Aplico umbral personalizado
threshold = 0.76
y_pred_threshold = (y_pred_proba >= threshold).astype(int)

# Después podemos evaluar con el nuevo umbral:
print("F1 :", f1_score(y_test, y_pred_threshold))
print("Recall :", recall_score(y_test, y_pred_threshold))
print("Precisión :", precision_score(y_test, y_pred_threshold))
print("Accuracy :", accuracy_score(y_test, y_pred_threshold))
print("Clasification Report:\n", classification_report(y_test, y_pred_threshold))

F1 : 0.5725028199022434
Recall : 0.6954933008526187
Precisión : 0.4864749733759318
Accuracy : 0.7870121760849204
Clasification Report:
               precision    recall  f1-score   support

         0.0       0.91      0.81      0.86     12731
         1.0       0.49      0.70      0.57      3284

    accuracy                           0.79     16015
   macro avg       0.70      0.75      0.72     16015
weighted avg       0.82      0.79      0.80     16015



In [147]:
#Metricas con el conjunto balanceado
print("Mejor modelo CatBoost:")
print("F1 :", f1_score(y_valid_sub, final_catboost.predict(X_valid_sub)))
print("Recall :", recall_score(y_valid_sub, final_catboost.predict(X_valid_sub)))
print("Precision :", precision_score(y_valid_sub, final_catboost.predict(X_valid_sub)))
print("Accuracy :", accuracy_score(y_valid_sub, final_catboost.predict(X_valid_sub)))
print("Clasification Report:\n", classification_report(y_valid_sub, final_catboost.predict(X_valid_sub)))

print("\nMejor modelo XGBoost:")
print("F1 :", f1_score(y_valid_sub, xgb_final.predict(X_valid_sub)))
print("Recall :", recall_score(y_valid_sub, xgb_final.predict(X_valid_sub)))
print("Precision :", precision_score(y_valid_sub, xgb_final.predict(X_valid_sub)))
print("Accuracy :", accuracy_score(y_valid_sub, xgb_final.predict(X_valid_sub)))
print("Clasification Report:\n", classification_report(y_valid_sub, xgb_final.predict(X_valid_sub)))


Mejor modelo CatBoost:
F1 : 0.7552740511695177
Recall : 0.7434462444771723
Precision : 0.7674842894790188
Accuracy : 0.759106529209622
Clasification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.77      0.76     10185
         1.0       0.77      0.74      0.76     10185

    accuracy                           0.76     20370
   macro avg       0.76      0.76      0.76     20370
weighted avg       0.76      0.76      0.76     20370


Mejor modelo XGBoost:
F1 : 0.7362576817122272
Recall : 0.8528227785959744
Precision : 0.6477255779269202
Accuracy : 0.6945017182130584
Clasification Report:
               precision    recall  f1-score   support

         0.0       0.78      0.54      0.64     10185
         1.0       0.65      0.85      0.74     10185

    accuracy                           0.69     20370
   macro avg       0.72      0.69      0.69     20370
weighted avg       0.72      0.69      0.69     20370

